# JS: Touch Navigation

> JavaScript generator for touch-to-nav conversion: swipe, drag,
> momentum, and pinch-to-zoom.

In [ ]:
#| default_exp js.touch

In [ ]:
#| export
from typing import Tuple

from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds
from cjm_fasthtml_card_stack.core.constants import (
    TOUCH_SWIPE_THRESHOLD, TOUCH_MOMENTUM_MIN_VELOCITY,
    TOUCH_MOMENTUM_FRICTION, TOUCH_PINCH_THRESHOLD,
    TOUCH_VELOCITY_SAMPLES,
)

## generate_touch_nav_js

Converts touch gestures on the card stack container into navigation
button clicks and scale adjustments. Uses the Pointer Events API with
`setPointerCapture` so that events survive HTMX OOB DOM swaps mid-drag.

**Single-finger gestures:**

- *Simple swipe* — a quick flick that doesn't cover a full card height
  triggers one navigation step.
- *Touch-and-drag* — holding the finger down and dragging triggers one
  step each time the finger travels one focused-slot-height of distance.
  After direction lock, `setPointerCapture` pins events to the card stack
  element so DOM mutations underneath don't break tracking.
- *Momentum* — a fast swipe continues navigating after the finger lifts,
  decelerating via an exponential friction model.

**Two-finger gestures:**

- *Pinch-to-zoom* — maps to the card stack's `increaseScale` /
  `decreaseScale` functions.

Supports mode-based disabling via `disable_in_modes`.

In [ ]:
#| export
def generate_touch_nav_js(
    ids: CardStackHtmlIds,  # HTML IDs for this card stack instance
    button_ids: CardStackButtonIds,  # Button IDs for navigation triggers
    disable_in_modes: Tuple[str, ...] = (),  # Mode names where touch nav is suppressed
) -> str:  # JavaScript code fragment for touch navigation
    """Generate JS for touch gesture to navigation conversion."""
    # Build mode check (same pattern as scroll.ipynb)
    if disable_in_modes:
        modes_array = ', '.join(f"'{m}'" for m in disable_in_modes)
        mode_check = f"""
        function isTouchDisabled() {{
            if (typeof window.kbNav !== 'undefined') {{
                const state = window.kbNav.getState();
                const disabledModes = [{modes_array}];
                return state && disabledModes.includes(state.currentMode);
            }}
            return false;
        }}
        """
        mode_guard = "if (isTouchDisabled()) return;"
        momentum_mode_guard = (
            "if (typeof isTouchDisabled === 'function' && isTouchDisabled()) "
            "{ _touchState.momentumId = null; return; }"
        )
    else:
        mode_check = ""
        mode_guard = ""
        momentum_mode_guard = ""

    return f"""
        // === Touch Navigation ===
        // Uses Pointer Events + setPointerCapture so that events survive
        // HTMX OOB DOM swaps that replace the original touch target mid-drag.
        const _touchState = {{
            pointers: new Map(),
            primaryId: null,
            active: false,
            startY: 0,
            startX: 0,
            lastY: 0,
            lastStepY: 0,
            stepDistance: 100,
            isNavigating: false,
            isPinching: false,
            pinchStartDist: 0,
            stepsTriggered: 0,
            history: [],
            momentumId: null,
            momentumAccum: 0,
        }};
        const _TOUCH_SWIPE_THRESHOLD = {TOUCH_SWIPE_THRESHOLD};
        const _TOUCH_MOMENTUM_MIN_VEL = {TOUCH_MOMENTUM_MIN_VELOCITY};
        const _TOUCH_MOMENTUM_FRICTION = {TOUCH_MOMENTUM_FRICTION};
        const _TOUCH_PINCH_THRESHOLD = {TOUCH_PINCH_THRESHOLD};
        const _TOUCH_VEL_SAMPLES = {TOUCH_VELOCITY_SAMPLES};
        {mode_check}
        function _getTouchStepDistance() {{
            const slot = document.querySelector(
                '#' + CSS.escape('{ids.card_stack}') + ' .viewport-slot[tabindex=\\\"0\\\"]'
            );
            if (slot) {{
                const h = slot.getBoundingClientRect().height;
                if (h > 0) return h;
            }}
            return 100;
        }}

        function _getPinchDistance() {{
            if (_touchState.pointers.size < 2) return 0;
            const pts = Array.from(_touchState.pointers.values());
            const dx = pts[0].x - pts[1].x;
            const dy = pts[0].y - pts[1].y;
            return Math.sqrt(dx * dx + dy * dy);
        }}

        function _stopMomentum() {{
            if (_touchState.momentumId) {{
                cancelAnimationFrame(_touchState.momentumId);
                _touchState.momentumId = null;
            }}
        }}

        function _fireTouchNav(direction) {{
            const btnId = direction === 'down'
                ? '{button_ids.nav_down}' : '{button_ids.nav_up}';
            const btn = document.getElementById(btnId);
            if (btn) btn.click();
        }}

        function setupTouchNavigation() {{
            const cardStack = document.getElementById('{ids.card_stack}');
            if (!cardStack || cardStack._touchNavSetup) return;
            cardStack._touchNavSetup = true;

            cardStack.addEventListener('pointerdown', function(evt) {{
                if (evt.pointerType !== 'touch') return;
                {mode_guard}
                _stopMomentum();

                _touchState.pointers.set(evt.pointerId, {{ x: evt.clientX, y: evt.clientY }});

                if (_touchState.pointers.size === 2) {{
                    // Second finger — switch to pinch mode
                    _touchState.isPinching = true;
                    _touchState.isNavigating = false;
                    _touchState.active = false;
                    _touchState.pinchStartDist = _getPinchDistance();
                    // Capture both pointers to survive DOM changes
                    for (const id of _touchState.pointers.keys()) {{
                        try {{ cardStack.setPointerCapture(id); }} catch (e) {{}}
                    }}
                    evt.preventDefault();
                    return;
                }}

                if (_touchState.pointers.size === 1) {{
                    _touchState.primaryId = evt.pointerId;
                    _touchState.active = true;
                    _touchState.startY = evt.clientY;
                    _touchState.startX = evt.clientX;
                    _touchState.lastY = evt.clientY;
                    _touchState.lastStepY = evt.clientY;
                    _touchState.isNavigating = false;
                    _touchState.isPinching = false;
                    _touchState.stepsTriggered = 0;
                    _touchState.history = [];
                    _touchState.stepDistance = _getTouchStepDistance();
                }}
            }});

            cardStack.addEventListener('pointermove', function(evt) {{
                if (evt.pointerType !== 'touch') return;
                {mode_guard}

                // Update tracked pointer position
                if (_touchState.pointers.has(evt.pointerId)) {{
                    _touchState.pointers.set(evt.pointerId, {{ x: evt.clientX, y: evt.clientY }});
                }}

                // --- Pinch mode ---
                if (_touchState.isPinching && _touchState.pointers.size >= 2) {{
                    evt.preventDefault();
                    const dist = _getPinchDistance();
                    const delta = dist - _touchState.pinchStartDist;
                    if (Math.abs(delta) >= _TOUCH_PINCH_THRESHOLD) {{
                        if (delta > 0) {{
                            if (ns.increaseScale) ns.increaseScale();
                        }} else {{
                            if (ns.decreaseScale) ns.decreaseScale();
                        }}
                        _touchState.pinchStartDist = dist;
                    }}
                    return;
                }}

                // --- Single-finger drag ---
                if (!_touchState.active || evt.pointerId !== _touchState.primaryId) return;

                const deltaY = evt.clientY - _touchState.startY;
                const deltaX = evt.clientX - _touchState.startX;

                // Direction lock: decide vertical vs horizontal
                if (!_touchState.isNavigating) {{
                    const totalDist = Math.abs(deltaY) + Math.abs(deltaX);
                    if (totalDist < 10) return;
                    if (Math.abs(deltaX) > Math.abs(deltaY)) {{
                        // Horizontal — abort touch nav
                        _touchState.active = false;
                        return;
                    }}
                    _touchState.isNavigating = true;
                    // Capture pointer on the card stack so events survive
                    // HTMX OOB swaps that replace elements under the finger
                    try {{ cardStack.setPointerCapture(evt.pointerId); }} catch (e) {{}}
                }}

                evt.preventDefault();

                // Velocity tracking via history buffer
                _touchState.history.push({{ t: evt.timeStamp, y: evt.clientY }});
                if (_touchState.history.length > _TOUCH_VEL_SAMPLES) {{
                    _touchState.history.shift();
                }}
                _touchState.lastY = evt.clientY;

                // Step threshold: one navigation per focused-slot-height
                const stepDelta = evt.clientY - _touchState.lastStepY;
                if (Math.abs(stepDelta) >= _touchState.stepDistance) {{
                    // Finger up (negative delta) = nav_down (next card)
                    const dir = stepDelta < 0 ? 'down' : 'up';
                    _fireTouchNav(dir);
                    _touchState.lastStepY += (stepDelta < 0 ? -1 : 1) * _touchState.stepDistance;
                    _touchState.stepsTriggered++;
                }}
            }});

            cardStack.addEventListener('pointerup', function(evt) {{
                if (evt.pointerType !== 'touch') return;
                _touchState.pointers.delete(evt.pointerId);

                // --- Pinch ending ---
                if (_touchState.isPinching) {{
                    if (_touchState.pointers.size < 2) {{
                        _touchState.isPinching = false;
                        if (_touchState.pointers.size === 1) {{
                            // One finger remains — reset to single-touch tracking
                            const remaining = _touchState.pointers.entries().next().value;
                            _touchState.primaryId = remaining[0];
                            _touchState.active = true;
                            _touchState.startY = remaining[1].y;
                            _touchState.startX = remaining[1].x;
                            _touchState.lastY = remaining[1].y;
                            _touchState.lastStepY = remaining[1].y;
                            _touchState.isNavigating = false;
                            _touchState.stepsTriggered = 0;
                            _touchState.history = [];
                            _touchState.stepDistance = _getTouchStepDistance();
                        }} else {{
                            _touchState.active = false;
                        }}
                    }}
                    return;
                }}

                if (!_touchState.active || evt.pointerId !== _touchState.primaryId) return;
                _touchState.active = false;

                if (!_touchState.isNavigating) return;

                // Compute velocity from history buffer
                let velocity = 0;
                const hist = _touchState.history;
                if (hist.length >= 2) {{
                    const first = hist[0];
                    const last = hist[hist.length - 1];
                    const dt = Math.max(1, last.t - first.t);
                    velocity = (last.y - first.y) / dt;
                }}

                // Simple swipe: no drag steps triggered but enough distance
                if (_touchState.stepsTriggered === 0) {{
                    const totalDelta = _touchState.lastY - _touchState.startY;
                    if (Math.abs(totalDelta) >= _TOUCH_SWIPE_THRESHOLD) {{
                        _fireTouchNav(totalDelta < 0 ? 'down' : 'up');
                    }}
                    return;
                }}

                // Momentum: continue navigating with deceleration
                const absVel = Math.abs(velocity);
                if (absVel >= _TOUCH_MOMENTUM_MIN_VEL) {{
                    const dir = velocity < 0 ? 'down' : 'up';
                    let curVel = absVel;
                    let lastFrame = performance.now();
                    const stepDist = _touchState.stepDistance;
                    _touchState.momentumAccum = 0;

                    function momentumTick(now) {{
                        {momentum_mode_guard}
                        const dt = now - lastFrame;
                        lastFrame = now;
                        // Time-normalized friction: consistent across frame rates
                        curVel *= Math.pow(_TOUCH_MOMENTUM_FRICTION, dt / 16);
                        _touchState.momentumAccum += curVel * dt;

                        if (_touchState.momentumAccum >= stepDist) {{
                            _touchState.momentumAccum -= stepDist;
                            _fireTouchNav(dir);
                        }}

                        if (curVel >= _TOUCH_MOMENTUM_MIN_VEL * 0.1) {{
                            _touchState.momentumId = requestAnimationFrame(momentumTick);
                        }} else {{
                            _touchState.momentumId = null;
                        }}
                    }}

                    _touchState.momentumId = requestAnimationFrame(momentumTick);
                }}
            }});

            cardStack.addEventListener('pointercancel', function(evt) {{
                if (evt.pointerType !== 'touch') return;
                _touchState.pointers.delete(evt.pointerId);
                if (_touchState.pointers.size === 0) {{
                    _touchState.active = false;
                    _touchState.isNavigating = false;
                    _touchState.isPinching = false;
                    _stopMomentum();
                }}
            }});
        }}

        // Expose for master coordinator
        ns._setupTouchNav = setupTouchNavigation;
    """

In [ ]:
# Test touch nav JS generation
from cjm_fasthtml_card_stack.core.html_ids import CardStackHtmlIds
from cjm_fasthtml_card_stack.core.button_ids import CardStackButtonIds

ids = CardStackHtmlIds(prefix="cs0")
btn = CardStackButtonIds(prefix="cs0")
js = generate_touch_nav_js(ids, btn)

# Element IDs and button IDs present
assert ids.card_stack in js
assert btn.nav_up in js
assert btn.nav_down in js

# No mode check when no modes specified
assert "isTouchDisabled" not in js

# Setup function exposed on namespace
assert "ns._setupTouchNav" in js

# Pointer Events API (not Touch Events)
assert "pointerdown" in js
assert "pointermove" in js
assert "pointerup" in js
assert "pointercancel" in js
assert "pointerType" in js
assert "setPointerCapture" in js

# Pointer tracking map for multi-touch
assert "pointers" in js
assert "primaryId" in js

# Pinch-to-zoom maps to scale controls
assert "ns.increaseScale" in js
assert "ns.decreaseScale" in js

# Momentum with requestAnimationFrame and friction
assert "_TOUCH_MOMENTUM_FRICTION" in js
assert "requestAnimationFrame" in js
assert "momentumTick" in js

# Direction locking (horizontal vs vertical)
assert "deltaX" in js

# Step distance from focused slot
assert "_getTouchStepDistance" in js
assert "viewport-slot" in js

# Constants inlined
assert "_TOUCH_SWIPE_THRESHOLD" in js
assert "_TOUCH_PINCH_THRESHOLD" in js
assert "_TOUCH_VEL_SAMPLES" in js

# Velocity history buffer
assert "history" in js

print("Touch nav JS basic tests passed!")

Touch nav JS basic tests passed!


In [ ]:
# Test with disabled modes
js_modes = generate_touch_nav_js(ids, btn, disable_in_modes=("split", "edit"))
assert "isTouchDisabled" in js_modes
assert "'split'" in js_modes
assert "'edit'" in js_modes
# Momentum loop also checks mode
assert "isTouchDisabled" in js_modes
print("Touch nav JS mode disabling tests passed!")

Touch nav JS mode disabling tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()